Задание:

Ручной ввод данных - бич любого проекта. Доводится сталкиваться с проектами, где даже дата рождения пациента не проходит ФЛК (мне попадалась запись о женщине, которая родилась 29 февраля 1995 года, хотя в тот год 29 февраля не было).
Но это именно то, с чем приходится работать.

Раз уж сейчас коронавирусная истерия, вот и задачка про коронавирус.
Есть база пациентов (они фиктивные, разумеется), есть их телефоны и результаты тестов. Нужно по 
всем положительным или сомнительным подтянуть телефоны. В ФИО и дате рождения пациента осознанно сделаны 
ошибки, но записи о людях всё-равно надо суметь сопоставить. При этом у человека может быть несколько 
телефонов и нужно найти все телефоны по человеку (даже если в конкретной записи про человека проба отрицательная, этот телефон может быть полезен, чтобы дозвониться до человека).

Размер базы уже такой, что думать про оптимизацию (по крайней мере - отсутствие избыточности) придётся.
Задачка предельно боевая, по нескольку раз в день приходится что-то подобное делать.

* [Сразу к итогу работы](#1)

In [1]:
import pandas as pd
import time

In [2]:
df = pd.read_csv('test_dataset.csv', sep=';')

In [3]:
df.head()

,lastname,firstname,patronname,birthday,phone,probe
0,Lrjwxmomnr,Hzkxwwe,Cyms,02.06.1939,3253848876,?
1,Fkekpvub,Tcwozktfxwip,Xjymccainppw,23.03.2019,5856232,-
2,Dslhomol,Ylkyxi,Dfryhks,20.07.1931,820576357,-
3,Hrfctwrxeo,Rxzzvklpm,Bfmdh,04.07.1927,6963367826,-
4,Pbqrr,Hcxavic,Tieahchbag,04.06.1983,9132573105,-


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2221094 entries, 0 to 2221093
Data columns (total 6 columns):
lastname      object
firstname     object
patronname    object
birthday      object
phone         int64
probe         object
dtypes: int64(1), object(5)
memory usage: 101.7+ MB


In [5]:
df.describe(include= object)

,lastname,firstname,patronname,birthday,probe
count,2221094,2221094,2065173,2132724,2221094
unique,1131128,1166849,1189271,116698,3
top,Qyii,Fbkn,Hjnz,27.03.1949,-
freq,15,16,16,112,1664897


In [6]:
# У нас есть пациент у которого брали пробу 15 раз. Необходимо уточнить причину по корой ему сделали так много 
# анализов. Также дата рождения 27.03.1949 оказалась очень популярной (встречается 112 раз)

<div style="border:solid red 4px; padding: 20px"> Почему повтор фамилии и имени говорит о том, что это один человек? Может же быть 5 Ивановых Ивана?.</div>

In [7]:
df.columns

Index(['lastname', 'firstname', 'patronname', 'birthday', 'phone', 'probe'], dtype='object')

In [8]:
# в качестве потготовки данных приведем поля lastname, firstname, patronname к нижнему регистру

In [9]:
df.lastname = df.lastname.str.lower()
df.patronname = df.patronname.str.lower()
df.firstname = df.firstname.str.lower()

In [10]:
df.shape

(2221094, 6)

In [11]:
# убедимся, что замена на нижний регистр прошла успешно

In [12]:
df.head()

,lastname,firstname,patronname,birthday,phone,probe
0,lrjwxmomnr,hzkxwwe,cyms,02.06.1939,3253848876,?
1,fkekpvub,tcwozktfxwip,xjymccainppw,23.03.2019,5856232,-
2,dslhomol,ylkyxi,dfryhks,20.07.1931,820576357,-
3,hrfctwrxeo,rxzzvklpm,bfmdh,04.07.1927,6963367826,-
4,pbqrr,hcxavic,tieahchbag,04.06.1983,9132573105,-


In [13]:
# проверим какие бывают пробы теста

In [14]:
df.probe.unique()


array(['?', '-', '+'], dtype=object)

In [15]:
# удачно, что отсутсвуют ошибочно введенные символы, отличающиеся от заданных

In [16]:
# Посмотрим на количество уникальных имен и фамилий

In [17]:
len(set(df.lastname))


1129192

In [18]:
len(df.firstname.unique())


1164302

In [19]:
# настораживает большое количество уникальных имен. Делаем вывод, что вероятно датасет искуственно созданный, 
# а не получен из реального с помощью обезличивания

In [20]:
#объединим имена и фамилии в одно множество

In [21]:
union_lastname_firstname = set(df.lastname) | set(df.firstname)

In [22]:
len(union_lastname_firstname)

2264299

<div style="border:solid red 4px; padding: 20px"> Исправила в нижнем комменте ошибку грамматическую</div>

In [23]:
# Проверим бывает ли, что перепутали имя и фамилию. Для этого достаточно получить пересечение Имен и Фамилий

In [24]:
intersept_lastname_firstname = set(df.lastname) & set(df.firstname)

In [25]:
len(intersept_lastname_firstname)

29195

In [26]:
len(set(df.lastname).intersection(set(df.firstname)))

29195

In [27]:
# Значит есть как минимум 29195 записей, в которых ошибочно занесенны Имена в графу Фамилия. 
# Для решение данной проблемы на реальном Датасете возможно путем поиска словаря имен на русском языке. И 
# последующей проверкой Фамилий из Датасета в словаре Имен русского языка. 


In [28]:
# Для решения данной задачи сначала подберем найдем способ перебора всех строк Датафрейма 
# и убедимся, что он работает с достаточной скоростью

In [29]:
start_time = time.time()
count = 0

for row_namedtuple in df.itertuples():
    count += 1 
    
print(count)
print("Прошло: {} секунд".format(time.time() - start_time))

2221094
Прошло: 18.148587942123413 секунд


In [30]:
# 2 секунды это приемлимо, а значит скорость достаточная, так что можно смело использовать

In [31]:
df.columns


Index(['lastname', 'firstname', 'patronname', 'birthday', 'phone', 'probe'], dtype='object')

In [32]:
# Исправим ошибку по записи Фамилии в поле firstname.

# Для этого создадим множество из значений поля firstname
# Проверим какое количество записей имеет совпадающие Имя и Фамилию и посчитаем их количество в переменной count_equal
#Для тех фамилий, которые содержаться в словаре Имен поменяем местами Имя и Фамилию
# теперь заменим поля firstname на lastname в тех именах, которые содержаться во множестве Фамилий 
# посчитаем количество 

In [33]:
set_firstname = set(df.firstname)
start_time = time.time()
count_1 = 0
count_2 = 0
count_equal = 0

firstname_itertuples = []
lastname_itertuples = []

for row_namedtuple in df.itertuples():
    
    if row_namedtuple.lastname == row_namedtuple.firstname:
        count_equal +=1 
        firstname_itertuples.append(row_namedtuple.firstname)
        lastname_itertuples.append(row_namedtuple.lastname)
        
    elif row_namedtuple.lastname in set_firstname:
        count_1 += 1
        firstname_itertuples.append(row_namedtuple.lastname)
        lastname_itertuples.append(row_namedtuple.firstname)
    else:
        count_2 +=1
        firstname_itertuples.append(row_namedtuple.firstname)
        lastname_itertuples.append(row_namedtuple.lastname)
    

print('count, count_1, count_2', count, count_1, count_2)
df.firstname = firstname_itertuples
df.lastname = lastname_itertuples

print("Прошло: {} секунд".format(time.time() - start_time))
print('Количество записей, в которых совпадало Имя и Фамилия (ошибка) равно ', count_equal)
print('Количество записей, в которых Фамилия содержалась во множестве Имен (ошибка) равно', count_1)
print('Количество записей, в которых фамилия не содержалась во множестве имен (нет ошибки)', count_2)



count, count_1, count_2 2221094 64534 2156560
Прошло: 20.51832604408264 секунд
Количество записей, в которых совпадало Имя и Фамилия (ошибка) равно  0
Количество записей, в которых Фамилия содержалась во множестве Имен (ошибка) равно 64534
Количество записей, в которых фамилия не содержалась во множестве имен (нет ошибки) 2156560


In [34]:
#  Исправлено 64534 записей, что можно считать хорошим результатом

In [35]:
# Проверим остались ли Имена, которые содержатся во множестве Фамилий

In [36]:
len(set(firstname_itertuples) & set(lastname_itertuples))

6978

<div style="border:solid red 4px; padding: 20px"> Исправила в нижнем комменте ошибки грамматические</div>

 К сожалению, у нас есть почти 7 тысяч имен, которые встречаются в фамилиях и при этом эти фамилии встречаются в именах, а значит их нельзя исправить простой перестановкой (перестановка уже сделана)...
 Здесь можно предположить, что сами данные не идеальные и дальше их улучшить в этой части на основании имеющихся данных невозможно.
 Удалять их не будем, поскольку даже, если Пациент с положительной пробой указан как Петров Иванов, но у него есть номер телефона, то с ним нужно связаться

In [37]:
intersept_lastname_firstname = set(df.lastname) & set(df.firstname)

In [38]:
len(intersept_lastname_firstname)

6978

In [39]:
# тут мы , убедились, что пересекать множества можно разными методами и одним и тем же результатом

In [40]:
# теперь поработаем с датой рождения

In [41]:
import datetime

In [42]:
df.head()

,lastname,firstname,patronname,birthday,phone,probe
0,lrjwxmomnr,hzkxwwe,cyms,02.06.1939,3253848876,?
1,fkekpvub,tcwozktfxwip,xjymccainppw,23.03.2019,5856232,-
2,dslhomol,ylkyxi,dfryhks,20.07.1931,820576357,-
3,hrfctwrxeo,rxzzvklpm,bfmdh,04.07.1927,6963367826,-
4,pbqrr,hcxavic,tieahchbag,04.06.1983,9132573105,-


In [43]:
# изменим тип на datetime

In [44]:
df['birthday_datetime'] = pd.to_datetime(df['birthday'], format='%d.%m.%Y', errors='coerce')

In [45]:
# Для развлечения добавим поле год рождения. 

In [47]:
df['birthday_datetime_year'] = df['birthday_datetime'].dt.year

In [48]:
# для визуального анализа посмотрим 10 случайных строк датафрейма

In [49]:
df.sample(10)

,lastname,firstname,patronname,birthday,phone,probe,birthday_datetime,birthday_datetime_year
1842948,rzrvwdcoklw,kgpio,tmwtxdzpq,06.10.2019,7245654712,-,2019-10-06,2019.0
1067683,yafnpw,mgmqhfnkprtl,ouuwclc,25.11.1966,6171161740,-,1966-11-25,1966.0
984650,tfvcqhjye,tmvi,onnzfogfvv,23.06.1975,8547428770,-,1975-06-23,1975.0
549806,rsipg,ejak,fwydg,15.12.1964,1915352,-,1964-12-15,1964.0
194554,hjhbjccnzx,iqemiteq,emzfmqa,17.04.1982,5319551,-,1982-04-17,1982.0
1660581,rqhmkyr,zrmtehbs,dcorirhy,05.01.1952,6098492,+,1952-01-05,1952.0
2126518,gyiijas,fbkkfxlcubej,yvicp,22.05.1949,292531,-,1949-05-22,1949.0
1631926,bchovbi,jioqsjnv,byfzphpzc,13.09.2015,8866829892,?,2015-09-13,2015.0
15062,oellncxwffl,gotc,ctazmz,01.05.1948,7527693317,?,1948-05-01,1948.0
148029,dpckkp,zqolhcpoo,kzqa,22.03.2011,7099536,-,2011-03-22,2011.0


In [50]:
# Все ОК

In [51]:
# Теперь на количество пропущенных значений 

In [52]:
df.isnull().sum() 


lastname                       0
firstname                      0
patronname                155921
birthday                   88370
phone                          0
probe                          0
birthday_datetime         162346
birthday_datetime_year    162346
dtype: int64

<div style="border:solid red 4px; padding: 20px"> Исправила в нижнем комменте ошибки </div>

In [53]:
# В поле patronname пропущено более 5 процентов данных (155921 пропусков). Предлагаю обсудить, что означает 
# это поле и почему так много пропусков и принять решение, о дальнейших действиях.

# birthday изначально было не заполнено 88370 записей 

# birthday_datetime в данном поле 162346 пропусков, а значит 77 тыс записей мы не смогли преобразовать в дату, 
# а значит в них содержались несуществующие даты.
# Я принимаю решение не пытаться исправить эти ошибки. При необходимости могу занятся данным вопросом.


In [54]:
import time

In [55]:
# Приступаем к решению задачи по составлению словаря в котором в качестве ключа будет id пациентов у которых есть 
# хотя бы дна неотрицательная проба, в качестве значения массив из всех указанных телефонов. Если телефон 
# повторяется, то мы все равно его будем добавлять

#Для решения данной задачи мы разделим всех пациентов на две части:
#    1) пациенты у которых заполнено поле birthday_datetime
#    2) пациенты у которых не заполнено поле birthday_datetime


<div style="border:solid red 4px; padding: 20px"> Снизу что за закомментная строка? На удаление?</div>

In [56]:
#del df['id']

In [57]:
df.birthday_datetime.isnull().sum()

162346

In [58]:
df.isnull().sum()

lastname                       0
firstname                      0
patronname                155921
birthday                   88370
phone                          0
probe                          0
birthday_datetime         162346
birthday_datetime_year    162346
dtype: int64

In [59]:
# Убедимся, что id указывается корректно

In [60]:
df_not_null = df.dropna(subset = ['birthday_datetime'], axis=0)

In [61]:
df_not_null.shape

(2058748, 8)

In [62]:
2058748 + 162346

2221094

In [63]:
df.shape

(2221094, 8)

In [64]:
# Количество строк в полученном df_not_null + количество строк с пропущенными значениями в birthday_datetime равно
# количеству строк в исходном датасете, значит все сделали корректно.

In [65]:
#del df_not_null['id']
df_not_null['id'] = df_not_null['lastname'] + ' ' + df_not_null['firstname'] + ' ' + df['birthday']

/Users/mariashemyakina/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [66]:
# проверим содержание id для второй строки

In [67]:
df_not_null.iloc[1][8]

'fkekpvub tcwozktfxwip 23.03.2019'

In [68]:
# посмотрим на сам ДатаФрейм у которого поле birthday_datetime без пропусков

In [69]:
df_not_null.head()

,lastname,firstname,patronname,birthday,phone,probe,birthday_datetime,birthday_datetime_year,id
0,lrjwxmomnr,hzkxwwe,cyms,02.06.1939,3253848876,?,1939-06-02,1939.0,lrjwxmomnr hzkxwwe 02.06.1939
1,fkekpvub,tcwozktfxwip,xjymccainppw,23.03.2019,5856232,-,2019-03-23,2019.0,fkekpvub tcwozktfxwip 23.03.2019
2,dslhomol,ylkyxi,dfryhks,20.07.1931,820576357,-,1931-07-20,1931.0,dslhomol ylkyxi 20.07.1931
3,hrfctwrxeo,rxzzvklpm,bfmdh,04.07.1927,6963367826,-,1927-07-04,1927.0,hrfctwrxeo rxzzvklpm 04.07.1927
4,pbqrr,hcxavic,tieahchbag,04.06.1983,9132573105,-,1983-06-04,1983.0,pbqrr hcxavic 04.06.1983


In [70]:
# возьмем датафрейм в корректно указанной birthday и его часть с неотрицательным результатом проб

In [71]:
df_not_null_poz_quest = df_not_null[(df_not_null['probe'] == '+') | (df_not_null['probe'] == '?')]

In [72]:
# Сначала сделаем словарь, в котором в качестве в качестве ключа id, а в качестве значений номера телефонов. 
# Пройдем по ДатаРФему df_not_null_poz_quest и будем добавлять дополнительный номер, если id уже в словаре
# в данном словаре, если id есть в словаре, то мы добавляем телефон даже если он уже есть у этого ключа. Таким 
# образом, если телефон указан дважды, то выше вероятность, что он без ошибки.
# В данном Датафрейме только неотрицательные результаты проб, поэтому все id номера телефонов нам нужны и мы 
# поместим их в словарь

In [73]:
df_not_null_poz_quest.shape

(515464, 9)

In [ ]:
slovar_not_null = {}
start_time = time.time()

for i in range(len(df_not_null_poz_quest)):
    if df_not_null_poz_quest.iloc[i][8] not in slovar_not_null:
        slovar_not_null[df_not_null_poz_quest.iloc[i][8]] = str(df_not_null_poz_quest.iloc[i][4])
    else:
        slovar_not_null[df_not_null_poz_quest.iloc[i][8]] += ', '
        slovar_not_null[df_not_null_poz_quest.iloc[i][8]] += str(df_not_null_poz_quest.iloc[i][4])
        
        
print("Time using: {} sec".format(time.time() - start_time))

In [ ]:
# время выполнения 244 сек вполне терпимое для данного датафрейма. Осталось обработать 1,5 млн записей, а значит 
# это займет 700 сек, т. е. менее 12 мин.

In [ ]:
print('Количество ключей в словаре равно', len(slovar_not_null))

In [ ]:
# Возьмем ДатаФрейм, в котором ненулевое значение birthday_datetime, но значение пробы указано как отрицательное

In [ ]:
df_not_null_negat = df_not_null[df_not_null['probe'] == '-']

In [ ]:
df_not_null_negat.shape

In [ ]:
# Добавим в словарь телефоны для тех ключей у которых есть неотрицательные пробы

In [ ]:
start_time = time.time()

for i in range(len(df_not_null_negat)):
    if df_not_null_negat.iloc[i][8] in slovar_not_null:
        slovar_not_null[df_not_null_negat.iloc[i][8]] += ', '
        slovar_not_null[df_not_null_negat.iloc[i][8]] += str(df_not_null_negat.iloc[i][4])
        
        
print("Time using: {} sec".format(time.time() - start_time))

In [ ]:
# добавили телефоны в словарь за 7.5 мин, что вполне нормально.

In [ ]:
# создадим ДатаФрейм из исходного в котором поле birthday не заполнено, либо заполнено с ошибками (в таком случае 
# незаполнено поле birthday_datetime)

In [ ]:
df_null = df[pd.isnull(df.birthday_datetime)]

In [ ]:
df_null.sample(10)

In [ ]:
df_null.shape

In [ ]:
#После визуального осмотра данных решаю в данном Датафрейме в качестве id взять lastname + firstname
# Можно еще добавить patronname, но есть риск, что в нем много ошибок и тогда станет только хуже

In [ ]:
df_null['id'] = df_null['firstname'] + ' ' + df_null['lastname']

In [ ]:
df_null.sample(5)

In [ ]:
#Визуальный осмотр данных удовлетворительный. Приступим к созданию словаря.
# Для этого разделим ДатаФрейм на 2:
#    1) С неотрицательными пробами
#    2) с отрицательными пробами

In [ ]:
df_null_poz_quest = df_null[(df_null['probe'] == '+') | (df_null['probe'] == '?')]

In [ ]:
df_null_negat = df_null[(df_null['probe'] == '-')]

In [ ]:
#Убедимся, что суммарно количество записей этих 2 ДатаФреймов равно исходному

In [ ]:
print('Размерность исходного Датафрейма', df_null.shape )

In [ ]:
print('Суммарное количество строк двух получившихся Датафреймов равно ', df_null_poz_quest.shape[0] + df_null_negat.shape[0])

In [ ]:
# все отлично, можно продолжать

In [ ]:
# Создадим словарь slovar_null, в который будем помещать id и номера телефонов пациентов у которых есть хотя бы  
# одна неотрицательная проба

In [ ]:
slovar_null = {}

In [ ]:
# Добавим в этот словарь id и номера телефонов для неотрицательных проб

In [ ]:
slovar_null = {}
start_time = time.time()

for i in range(len(df_null_poz_quest)):
    if df_null_poz_quest.iloc[i][8] not in slovar_null:
        slovar_null[df_null_poz_quest.iloc[i][8]] = str(df_null_poz_quest.iloc[i][4])
    else:
        slovar_null[df_null_poz_quest.iloc[i][8]] += ', '
        slovar_null[df_null_poz_quest.iloc[i][8]] += str(df_null_poz_quest.iloc[i][4])
        
        
print("Time using: {} sec".format(time.time() - start_time))

In [ ]:
# Теперь обогатим наш словарь телефонами из Датафрейма с отрицательными пробами df_null_negat.

In [ ]:
start_time = time.time()

for i in range(len(df_null_negat)):
    if df_null_negat.iloc[i][8] in slovar_null:
        slovar_null[df_null_negat.iloc[i][8]] += ', '
        slovar_null[df_null_negat.iloc[i][8]] += str(df_null_negat.iloc[i][4])
        
        
print("Time using: {} sec".format(time.time() - start_time))

 # Итог <a class="anchor" id="1"></a>



   Проведена работа по исправлению ошибки если Имя попало в Фамилию и наоборот. Однако в исходном датасете есть ошибки типа Имя + Имя и Фамилия + Фамилия, а их исправить в автоматическом режиме без словаря имен нельзя.

Мы получили 2 словаря: 

   - 1 словарь slovar_not_null содержит в качестве ключа  сумму 
        df_not_null['lastname'] + ' ' + df_not_null['firstname'] + ' ' + df['birthday']
        для всех записей, а в качестве значения, все телефоны (включая телефоны для отрицательных проб, если
        с для данного id есть хоть одна неотрицательная проба)
        В этом словаре содержаться информация по записям, в которых указанную дату удалось преобразовать в datetime
        
   - 2 словарь slovar_null содержит в качестве ключа содержит сумму 
        df_not_null['lastname'] + ' ' + df_not_null['firstname'] 
        для всех записей, а в качестве значения, все телефоны (включая телефоны для отрицательных проб, если
        с для данного id есть хоть одна неотрицательная проба)
        
 В Данном решении я пожертвовал тем, что если у пациента есть неотрицательная проба, в которой корректно указана дата рождения и есть отрицательные пробы  в которых дата рождения не указана, либо указана некорректно, то для меня это разные пациенты.
 
Общее время расчета 12 минут.